In [17]:
import os 
import sys

In [18]:
# Set path to parent dir to import personal imports
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from data.TrajectorySet import TrajectorySet
from data.Sampler import Sampler 
from utils.tensor_utils import convert_batch_to_tensor

import torch 
import torch.utils.data as data 
from torch.utils.data import Dataset

import minari 

In [19]:
minari_dataset = minari.load_dataset("D4RL/pointmaze/large-v2")

In [31]:
class DatasetCL(Dataset): 
    def __init__(self, sampler, batch_size: int, k: int = 2): 
        """
        sampler: The Sampler class to sample batches. 
        batch_size: The size of the batch (ie. the number of state pairs)
        k: A hyperparameter that dictates the average number of 
                positive pairs sampled from the same trajectory. The 
                lower the number, the lesser the chance of false negatives. 
        """
        self.sampler = sampler 
        self.batch_size = batch_size 
        self.k = k 

        anchor, positive = convert_batch_to_tensor(self.sampler.sample_batch(batch_size=self.batch_size, k=self.k))
        self.pairs = list(zip(anchor, positive))

    def __len__(self): 
        return len(self.pairs)
    
    def __getitem__(self, index):
        s_i, s_j = self.pairs[index] 
        return (torch.as_tensor(s_i, dtype=torch.float32), torch.as_tensor(s_j, dtype=torch.float32))
    
    def get_batch(self):
        return self.pairs

In [30]:
"""
TESTING CELL! 
"""

T = TrajectorySet(dataset=minari_dataset) 

S = Sampler(T, dist="u") 
ds = DatasetCL(sampler=S, batch_size=4096, k=2)


#batch = ds.get_batch()
#print(batch)
print(len(ds))
print(ds[0])


#DS = data.DataLoader(dataset=ds, batch_size=2, shuffle=True)

#for batch in DS: 
#    print(batch)




4096
(tensor([ 2.6791, -0.9943, -0.4335, -0.3731]), tensor([-0.6793,  0.9388, -3.9365, -0.2363]))
